# Light interception

In [ ]:
from utils import *
from oawidgets.plantgl import PlantGL
from openalea.plantgl.all import *
from openalea.rose.data import manips, experiments
from ipywidgets import interactive, Dropdown, Checkbox

from openalea.spice.simulator import *
from openalea.spice import Scene as spScene
from openalea.spice.common.convert import pgl_to_spice, spice_add_pgl

In [ ]:
def select_experiment(manip_name="Manip1_C1_PARfort", stage="P1", fill=True):
    pgl_scene = experiment_named(manip_name, stage, fill)
    env = environment()
    simulator = Simulator()
    simulator.configuration.NB_PHOTONS = int(1e7)
    simulator.configuration.SCALE_FACTOR = 10
    simulator.configuration.MAXIMUM_DEPTH = 50
    simulator.configuration.T_MIN = 0.01
    simulator.configuration.BACKFACE_CULLING = False
    simulator.configuration.KEEP_ALL = False
    

    # setup environmentphmap
    sp_scene = pgl_to_spice(pgl_scene, sensors=True, setup=False)
    sp_scene = spice_add_pgl(sp_scene, env, sensors=False, setup=True)
    for sh in pgl_scene:
        simulator.addFaceSensor(sh)
    
    simulator.scene_pgl = pgl_scene + env
    simulator.scene = sp_scene

    # S2 series of lamps
    simulator.addPointLight(Vec3(730, 1670, 2100), 400)
    simulator.addPointLight(Vec3(730, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 176, 2100), 400)
    simulator.addPointLight(Vec3(1670, 1660, 2100), 400)
    
    # run
    res = simulator.run()
    sc = simulator.visualizePhotons(mode="oawidgets")
    display(sc)
    return simulator

opts = list(map(str, list(range(len(get_all_expe())))))
manips_opts = sorted(list(map(str, list(manips()))))
stage_opts = ["P1","P2","P3"]

In [ ]:
manip_name=Dropdown(options=manips_opts, value="Manip1_C1_PARfort")
stage=Dropdown(options=stage_opts, value="P1")

def update_stages(*args):
    options = experiments(manip_name.value)
    options = [o.name for o in options] 
    stage.options = sorted(options)

manip_name.observe(update_stages, "value")
fill=Checkbox(value=False)

In [ ]:
w = interactive(select_experiment, {'manual': True}, manip_name=manip_name, stage=stage, fill=fill)
w

In [ ]:
sim = w.result